# Type Inference in Lettuce

Thus far, we introduced a type system that required us to provide annotations such as 

~~~
let y: num = 15 in 
   let f : num -> bool = function (x: num) x >= y in 
        f(y) 
~~~

Clearly, the type annotations are often annoying to provide since they are "obvious". For instance: 

~~~
let y: num = 15 in ...
~~~

It is clear that if we are assigning `y` to `15`, shouldn't its type be __num__. Shouldn't our
interpreter be smart enough to recognize that?

For instance, scala has some limited type inference that saves us from having to write types all over
the place.


In [1]:
val x = 15 // No need to say x: Int

x: Int = 15

In [3]:
def f(x: Int) = x + 25 // No  need to say that f returns an integer

defined function f

In [5]:
def f (x: Int) = List(x) // No need to say that f returns a list of integers

defined function f

Scala has a limited form of type inference, that tries its best to infer missing annotations. Generally, it requires us to declare arguments to a function but not necessarily the return types. The one exception to this is recursive functions.

In [6]:
def f (x: Int) = if (x <= 15) 
                    25
                  else 
                    x + 55

defined function f

In [0]:
def f(x: Int) = if (x <= 15)
                    25
                else
                    f(x-1) + 54

cmd0.sc:4: recursive method f needs result type
                    f(x-1) + 54
                    ^

: 

Our goal therefore, is to provide type inference in Lettuce. We will assume that the program needs to be typed as in the previous lecture. However, let us also assume that no types are annotated by the user.

### Example 1

~~~
let x: ? = 25 in 
    let y: ?  = 35 >= x in 
       let f : ? = function (z: ?)  if (y) 
                                    then z + x >= 10 
                                    else z <= 15
               in 
               f(35)
~~~

Since none of the types are provided, our goal is to infer the missing annotations here.  In some cases, the user may take pity on us and provide an annotation. However, we will assume that is not the case in this lecture and focus on *type inference*, assuming that no type annotations are given to us.

The goal of type inference is two fold:
1. Fill up the missing type information in the program.
2. Report type errors to indicate that certain parts of the program cannot be provided consistent types.


Thus, example 1 should be typed properly as


~~~
let x: num = 25 in 
    let y: bool  = 35 >= x in 
       let f : num -> bool = function (z: num)  if (y) 
                                    then z + x >= 10 
                                    else z <= 15
               in 
               f(35)
~~~


### Example 2

Consider the Lettuce program

~~~
let x: ? = 25 in 
   let f : ? = function (t: ?) 
                    t >= 35 
           in 
       f(45) >= x
~~~

This program leads to a type error. Why? 
- We are forced to assign type __num__ to x due to ` let x = 25`
- We are forced to assign type __num__ to t due to  `t >= 35`
- Thus, we are forced to annotate `f` as __num__ => __bool__
- Thus, `f(45)` call returns a __bool__
- However, the program now compared __bool__ and __num__ using >=, which is an error.

### Example 3

Let us take a more complex example

~~~
let f : ? = function (g: ?) 
               function (x: ?) 
                   g( g(x) )
       in 
   let double: ? = function (x) 2 * x in 
      let doubool: ? = function (b) b and b in 
         let y = f(double)(20)  in 
           let z = f(doubool) (false) in 
               y
~~~


This turns out to have a type error. The issue is that function `f` is called in 
two ways. In one its argument `g` has type `num => num` and `x` has type `num`. However, in the other
call argument `g` has the type `bool => bool` and `x` has type `bool`.

Therefore, function `f` and its parameters `g`, `x` are not going to receive types.

However, note that this program executes fine under our old dynamically typed interpreter. 
The type of f is *polymorphic*: (t => t) => t => t, where t can be num or bool. However,
our system does not (yet) allow polymorphic functions.
$$\newcommand\num{\mathbf{num}}$$
$$\newcommand\bool{\mathbf{bool}}$$
$$\newcommand\Ra{\Rightarrow}$$

## Type Inference: Overall Idea

Thus, the overall idea behind type inference is simple.
1. Setup type variables: We will start by setting up unknown type variables for every expression in the program. These type variables are going to stand in for unknown types.
2. Make up equations over the type variables.
3. Solve the equations to obtain the resulting types.


### 1. Setting up type variables.

The idea here is that we will make up a fresh type variable for every subexpression of the program. This is best illustrated using an example.

~~~
let x = 15 in 
   let y = x + 30 in 
       x + y
~~~

| Expression | Type Variable |
|-------------|---------------|
|    x        |   $t_x$       | 
|    y        |   $t_y$       |



In general, you can imagine walking down the AST and making a type expression for every node in it. This leads to a lot of variables and we will generally avoid doing so, when we get to implement this algorithm.

### 2. Setting up equations between type variables.

Next, we will have to systematically setup equations between type variables based on the program.

As in the example, 

- `let x = 15 in ...`, the type of x must equal that of 15.  We get the eq. $t_x = num$
- `let y = x + 30 .. `, the type of y must equal to that of x + 30, which leads us to conclude that
$t_x = num$. We will conclude that  $t_y = num$ since `x+30` yields a num.
- Finally, the expression `x+y` must have type num as must each of its arguments. Once again, we simply
obtain $t_x = num$ and $t_y = num$. 

The type inference simply needs to note that $t_x = t_y = \num$.


### Example 2

~~~
let x: ? = 25 in 
   let f : ? = function (t: ?) 
                    t >= 35 
           in 
       f(45) >= x
~~~

| Expression | Type Variable |
|-------------|---------------|
|    x        |   $t_x$       | 
|    f        |   $t_f$       |
|    t        |   $t_t$       |
|   f(45)     |   $t_c$       | 


What are the equations we generate?
$$ \begin{array}{lll}
\hline
ID & Equation & Remark\\
\hline
E1: & t_x = \num & \texttt{let x = 25 in .. }\\
E2: & t_f = t_t \Ra \bool & \texttt{let f = function (t) t >= 35 in .. }\\
E3: & t_t = \num & \texttt{t >= 35}\\
E4: & t_f = \num \Ra t_c & \texttt{f(45)} \\
E5: & t_c = \num & \texttt{f(45) >= x} \\
\end{array} $$

- $t_x = num$ is easy to see why.
- $t_f$ is a function, whose argument $t$ has assigned type $t_t$.  Thus, $t_f = t_t => bool$.
- Similarly, we obtain that $t_t = num$ since it is compared with $35$ in the body of $f$.
- How do we get the constraint $t_f = \num \Ra t_c$ from the function call `f(45)`. 
  - Note that $t_c$ is the type of the expression `f(45)` or `FunCall(Ident("f"), Const(45))` in abstract syntax.
  - The type of the argument to the call is `num`.
  - Therefore, the constraint says " the type of the called function must take as input the type of the argument and return the type of the overall expression "
- Finally, note that $f(45)$ is compared using `>=`. Its type $t_c$ must equal `num`.

### Solving the equations

Let us take these equations and try to solve them.

$$\begin{array}{rl}
t_x & = \num \\
t_t & = \num \\
t_c & = \num \\
t_f & = t_t \Ra \bool \\
t_f &= \num \Ra t_c \\
\end{array}$$

Note that we already have solutions for $t_x, t_t $ and $t_c$. Substituting, gives us:
$$\begin{array}{rl}
t_f & = \num \Ra \bool\\
t_f &= \num \Ra \num \\
\end{array}$$

Breaking down the two definitions of $t_f$ yields the fact that $t_f \not= t_f$ and this a __conflict__.
The system of type equations have no solution for this problem.

## Generating Equations.

To systematically generate equations, we create a type environment $\alpha$ that maps identifiers to their types. However, $\alpha(x)$ for an identifier $x$ can be a type expression that contains type variables. For instance, $\alpha(x):\ \num\ \Ra (\bool \Ra t_y)$.

To be formal, we will allow our definition of a type to also have type variables.

$$\begin{array}{rcl}
\mathbf{Type} & \Ra & \text{TypeVar}(\mathbf{String}) \\
& | & \text{NumType} \\
& | & \text{BoolType} \\
& | & \text{FunType}(\mathbf{Type}, \mathbf{Type}) \\
\end{array}$$

Note that our final goal is to annotate every unknown identifier with a type
that is __not__ a TypeVar. But to begin with, the TypeVar construct is introduced
to deal with unknowns.

We will specify the behavior of a function 

~~~
generateEquations(e: Expr, alpha: TypeEnvironment): (Type, ListOfEquations)
~~~

wherein 

~~~
type TypeEnvironment = Map[String, Type]

type ListOfEquations = List[(Type, Type)]

~~~


In [1]:
sealed trait Expr

case class Const(f: Double) extends Expr
case class Ident(x: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Geq (e1: Expr, e2: Expr) extends Expr
case class And(e1: Expr, e2: Expr) extends Expr
case class Not(e: Expr) extends Expr
case class IfThenElse(e: Expr, e1: Expr, e2: Expr) extends Expr
case class Let(x: String, e1: Expr, e2: Expr) extends Expr
case class FunDef(param: String, body: Expr) extends Expr
case class FunCall(e1: Expr, e2: Expr) extends Expr
case class LetRec(f: String, x: String, e1: Expr, e2: Expr ) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Geq
defined class And
defined class Not
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class LetRec

In [4]:
sealed trait Type
case object NumType extends Type
case object BoolType extends Type
case class FunType(type1: Type, type2: Type) extends Type
case class TypeVar(name:String) extends Type

defined trait Type
defined object NumType
defined object BoolType
defined class FunType
defined class TypeVar

In [9]:
type TypeEnvironment = Map[String, Type]
type ListOfEquations = List[(Type, Type)]

defined type TypeEnvironment
defined type ListOfEquations

In [25]:
object TypeVarGenerator {
    var counter = 0
    
    def getFreshTypeVariable(id: String): TypeVar = {
        val t = TypeVar("type_" + id + "_" + counter.toString)
        counter = counter + 1
        t
    }
    
    def resetCounter = {
        counter = 0
    }
}

defined object TypeVarGenerator


### Const, Ident

- An AST node `Const(f)` always has type __num__, regardless of its type environment.

~~~
def generateEquations(e: Expr, alpha: TypeEnvironment): (Type, ListOfEquations) = 
   e match {
      case Const(f) => (NumType, Nil) // If expr is a constant, return num and Nil
      ....
   }
~~~

- An AST node `Ident("x")` has type $\alpha(x)$ if $x \in \mathbf{domain}(\alpha)$. Otherwise,
if $x \not\in \mathbf{domain}(\alpha)$, we deduce that $x$ is being used before it is bound (typeerror).

~~~
    case Ident(id) => {
          if (alpha contains id){
              (alpha(id), Nil)
          } else {
              throw new ErrorException(s"Used undeclared identifier $id -- type error")
          }
      }
    
~~~


### Arithmetic, Comparison and Boolean

- AST node `Plus(e1, e2)`  (same for Minus, Mult, Divide, Log, Exp, Sine and Cosine) 
has type __num__. Furthermore, we will generate the constraints that `e1` and `e2` have
the type __num__.

~~~
    case Plus(e1, e2) => {
         val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
         val (t2, lst2) = generateEquations(e2, alpha) // Gen. eqs for e2
         // Combine the equations and to them add the 
         // extra equations t1 == num, t2 == num
         val combinedList = lst1 ++ lst2 ++ List( (t1, NumType), (t2, NumType) )
         (NumType, combinedList) // The overall type of Plus is a num
    }
~~~




- AST node `Geq(e1, e2)` has type __bool__ while we generate constraints that `e1`, `e2` have
type __num__.

~~~
    case Geq(e1, e2) => {
         val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
         val (t2, lst2) = generateEquations(e2, alpha) // Gen. eqs for e2
         // Combine the equations and to them add the 
         // extra equations t1 == num, t2 == num
         val combinedList = lst1 ++ lst2 ++ List( (t1, NumType), (t2, NumType) )
         (BoolType, combinedList) // Overall type of Geq is a boolean
    }
~~~


- AST node `IfThenElse(e, e1, e2)` requires that the type of `e` be Boolean and `e1, e2` have the same type.


~~~
    case IfThenElse(e, e1, e2) => {
         val (t0, lst0) = generateEquations(e, alpha) // Gen. eqs for e
         val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
         val (t2, lst2) = generateEquations(e2, alpha) // Gen. eqs for e2
         // Combine the equations and to them add the 
         // extra equations t0 == bool and t1 == t2
         val combinedList = lst1 ++ lst2 ++ List( (t0, BoolType), (t1, t2) )
         (t1, combinedList) // Overall type of IfThenElse is t1
    }
~~~

### Function Definition



- Consider a AST node  $e$ given by `FunDef(x, e1)`.  Let $t_x$ be the type of $x$ and $t_{e1}$ that of `e1`. We obtain the type equation: $t_e = t_x \Ra t_{e1}$.

~~~
    case FunDef(param, body) => {
        val tparam = getFreshTypeVariable() // Gen. fresh type variable 
        val newAlpha = alpha + (param -> tparam) // Generate a new type env.
        val (tbody, listBody) = generateEquations(body, newAlpha)
        val fnType = FunType(tparam, tbody)
        (fnType, listBody)
    }
~~~



###  Function Call

- Consider an AST node $e$ given by `FunCall(e1, e2)`.  
  - We will let $t_{e1}, t_{e2}$ be the unknown types of `e1`, `e2`. 
  - It follows that $t_{e1} = t_{e2} => t_e $ where $t_e$ is the type of the function call expression.
  
~~~
    case FunCall(e1, e2) => {
        val (te1, listE1) = generateEquations(e1, alpha)
        val (te2, listE2) = generateEquations(e2, alpha)
        val te = getFreshTypeVariable()
        
        val newTypeConstraint = (te1, FunType(te2, te) ) // te1 == te2 => te
        val combinedList = listE1 ++ listE2 ++ List(newTypeConstraint)
        
        (te, combinedList)
    }

~~~

Recursion is not hard and is left as an exercise for the reader.

### Let Bindings

For a let binding `let x = e1 in e2`, under type environment $\alpha$, we proceed as follows:
  - Create a fresh type variable $t_x$ for $x$.
  - Generate the list of equations for `e1` under environment $\alpha$. Let $t_1$ be the type of `e1`.
  - Add the equation $t_x == t_1$, thus equating the type of $x$ to that of `e1`.
  - Generate the list of equations for `e2` under the environment $\alpha$. Let $t_2$ be the type of `e2`.
  - The final type of the entire let binding is $t_2$ with the combined equations generated when processing `e1`, `e2` and the equation $t_x == t_1$.
 
~~~
   case Let(x, e1, e2) => {
       val tx = getFreshTypeVariable()
       val (te1, listE1) = generateEquations(e1, alpha)
       val newAlpha = alpha ++ (x -> tx)
       val (te2, listE2) = generateEquations(e2, newAlpha)
       val combinedList = listE1 ++ listE2 ++ List( tx, te1) 
       (te2, combinedList)
   }
~~~



In [28]:
case class ErrorException(s: String) extends Exception 

def generateEquations(e: Expr, alpha: TypeEnvironment): (Type, ListOfEquations) = 
e match {
    case Const(f) => (NumType, Nil) // If expr is a constant, return num and Nil
    case Ident(id) => {
          if (alpha contains id){
              (alpha(id), Nil)
          } else {
              throw new ErrorException(s"Used undeclared identifier $id -- type error")
          }
    }
    
    case Plus(e1, e2) => {
         val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
         val (t2, lst2) = generateEquations(e2, alpha) // Gen. eqs for e2
         // Combine the equations and to them add the 
         // extra equations t1 == num, t2 == num
         val combinedList = lst1 ++ lst2 ++ List( (t1, NumType), (t2, NumType) )
         (NumType, combinedList) // The overall type of Plus is a num
    }
    
    case Geq(e1, e2) => {
         val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
         val (t2, lst2) = generateEquations(e2, alpha) // Gen. eqs for e2
         // Combine the equations and to them add the 
         // extra equations t1 == num, t2 == num
         val combinedList = lst1 ++ lst2 ++ List( (t1, NumType), (t2, NumType) )
         (BoolType, combinedList) // Overall type of Geq is a boolean
    }
    
    case And(e1, e2) => {
        val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
        val (t2, lst2) = generateEquations(e2, alpha) // Gen. eqs for e2
        // Combine the equations and to them add the 
        // extra equations t1 == bool, t2 == bool
        val combinedList = lst1 ++ lst2 ++ List( (t1, BoolType), (t2, BoolType) )
        (BoolType, combinedList) // Overall type of And is a boolean
    }
    
    case Not(e1) => {
        val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
        val combinedList = lst1 ++ List( (t1, BoolType) )
        (BoolType, combinedList) // Overall type of And is a boolean
    }
    
    case IfThenElse(e, e1, e2) => {
         val (t0, lst0) = generateEquations(e, alpha) // Gen. eqs for e
         val (t1, lst1) = generateEquations(e1, alpha) // Gen. eqs for e1
         val (t2, lst2) = generateEquations(e2, alpha) // Gen. eqs for e2
         // Combine the equations and to them add the 
         // extra equations t0 == bool and t1 == t2
         val combinedList = lst1 ++ lst2 ++ List( (t0, BoolType), (t1, t2) )
         (t1, combinedList) // Overall type of IfThenElse is t1
    }
    
     case FunDef(param, body) => {
        val tparam = TypeVarGenerator.getFreshTypeVariable(param) // Gen. fresh type variable 
        val newEnv = alpha + (param -> tparam)
        val (tbody, listBody) = generateEquations(body, newEnv)
        val fnType = FunType(tparam, tbody)
        (fnType, listBody)
    }
    
    case FunCall(e1, e2) => {
        val (te1, listE1) = generateEquations(e1, alpha)
        val (te2, listE2) = generateEquations(e2, alpha)
        val te = TypeVarGenerator.getFreshTypeVariable("fcall")
        
        val newTypeConstraint = (te1, FunType(te2, te) ) // te1 == te2 => te
        val combinedList = listE1 ++ listE2 ++ List(newTypeConstraint)
        
        (te, combinedList)
    }
    
     case Let(x, e1, e2) => {
       val tx = TypeVarGenerator.getFreshTypeVariable(x)
       val (te1, listE1) = generateEquations(e1, alpha)
       val newAlpha = alpha + (x -> tx)
       val (te2, listE2) = generateEquations(e2, newAlpha)
       val combinedList = listE1 ++ listE2 ++ List( (tx, te1) )
       (te2, combinedList)
   }
    
   case LetRec(f, x, e1, e2) => ???
    
}

defined class ErrorException
defined function generateEquations

In [30]:
def typeToString(t: Type) : String = t match {
    case NumType => "num"
    case BoolType => "bool"
    case FunType(t1, t2) => "("+(typeToString(t1)) + " => " + (typeToString(t2)) +")"
    case TypeVar(str) => str
}

def prettyPrintTypeEqs (lst: List[(Type, Type)]): Unit = {
    lst.foreach {
        case (t1, t2) => {
            println( typeToString(t1) + " == " + typeToString(t2) )
        }
    }
}

defined function typeToString
defined function prettyPrintTypeEqs

### Examples

Let us try to generate the equations for some example programs now.

~~~
let f = function (x) x >= 15 in 
    f(35)
~~~

In [33]:
val x = Ident("x")
val f = Ident("f")
val fif = Const(15.0)
val thf = Const(35.0)
val prog = Let("f", FunDef("x", Geq(x, fif)), FunCall(f, thf) )

val emptyTypeEnv = Map[String, Type]()

TypeVarGenerator.resetCounter


val (typ, lstOfEqs) = try 
   generateEquations(prog, emptyTypeEnv)
catch {
    case ErrorException(msg) => { println(msg); throw new ErrorException(msg) }
}

println("-- Generated Eqs --")
prettyPrintTypeEqs(lstOfEqs)
println("The overall program has type: " + typeToString(typ))

-- Generated Eqs --
type_x_1 == num
num == num
type_f_0 == (num => type_fcall_2)
type_f_0 == (type_x_1 => bool)
The overall program has type: type_fcall_2


x: Ident = Ident("x")
f: Ident = Ident("f")
fif: Const = Const(15.0)
thf: Const = Const(35.0)
prog: Let = Let(
  "f",
  FunDef("x", Geq(Ident("x"), Const(15.0))),
  FunCall(Ident("f"), Const(35.0))
)
emptyTypeEnv: Map[String, Type] = Map()
typ: Type = TypeVar("type_fcall_2")
lstOfEqs: ListOfEquations = List(
  (TypeVar("type_x_1"), NumType),
  (NumType, NumType),
  (TypeVar("type_f_0"), FunType(NumType, TypeVar("type_fcall_2"))),
  (TypeVar("type_f_0"), FunType(TypeVar("type_x_1"), BoolType))
)

Now consider the ill typed program:
~~~
let f = function (x) x >= 30 in 
    f(10) + 25
~~~



In [35]:
val x = Ident("x")
val f = Ident("f")
val thirty = Const(30.0)
val ten = Const(10.0)
val twf = Const(25.0)
val prog = Let("f", FunDef("x", Geq(x, thirty)), Plus(FunCall(f, ten), twf ))

val emptyTypeEnv = Map[String, Type]()

TypeVarGenerator.resetCounter


val (typ, lstOfEqs) = try 
   generateEquations(prog, emptyTypeEnv)
catch {
    case ErrorException(msg) => { println(msg); throw new ErrorException(msg) }
}

println("-- Generated Eqs --")
prettyPrintTypeEqs(lstOfEqs)
println("The overall program has type: " + typeToString(typ))

-- Generated Eqs --
type_x_1 == num
num == num
type_f_0 == (num => type_fcall_2)
type_fcall_2 == num
num == num
type_f_0 == (type_x_1 => bool)
The overall program has type: num


x: Ident = Ident("x")
f: Ident = Ident("f")
thirty: Const = Const(30.0)
ten: Const = Const(10.0)
twf: Const = Const(25.0)
prog: Let = Let(
  "f",
  FunDef("x", Geq(Ident("x"), Const(30.0))),
  Plus(FunCall(Ident("f"), Const(10.0)), Const(25.0))
)
emptyTypeEnv: Map[String, Type] = Map()
typ: Type = NumType
lstOfEqs: ListOfEquations = List(
  (TypeVar("type_x_1"), NumType),
  (NumType, NumType),
  (TypeVar("type_f_0"), FunType(NumType, TypeVar("type_fcall_2"))),
  (TypeVar("type_fcall_2"), NumType),
  (NumType, NumType),
  (TypeVar("type_f_0"), FunType(TypeVar("type_x_1"), BoolType))
)


## Systematically solving Equations using Unification

The main idea is to solve the equations systematically using unification. Let us take an example.

~~~
let f : ? = function (x: ?) 
                   x >= 30
       in 
   f(10) + 25 
~~~

### Step 1: Make the type variables for various expressions.

$$\begin{array}{rl}
\hline
\text{Expr.}/\text{Identifier} & \text{Type} \\
\hline
\texttt{f} & t_f \\
\texttt{function (x: ?)  x >= 30} & t_r\\
\texttt{x} & t_x \\
\texttt{x >= 30} & \bool \\
\texttt{f(10) + 25} & \num \\
\texttt{f(10)} & t_c \\
\hline
\end{array}$$

### Step 2: Generate Equations.

$$\begin{array}{rl}
\hline
\text{Source} & \text{Eqn.} \\
\hline
\texttt{let f = function .. } & t_f =  t_r\\
\texttt{function (x) x >= 30} & t_r = t_x \Ra \bool \\
\texttt{x >= 30} & t_x = \num \\
\texttt{f(10) + 25} & t_c = \num \\
\texttt{f(10)} & t_f = \num \Ra t_c  \\
\hline
\end{array}$$

### Step 3: Solve Equations.

Let us consider an algorithm for solving equations. The idea is simple: we will systematically process equations using rules. One of the rules is to convert an equation of the form $t = \textit{rhs type expression}$ into a
substitution rule that says whenever you see $t$, replace it with the $\textit{rhs type expression}$.

To begin with we have all the equations and no substitutions.

$$\begin{array}{rl}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
t_f =  t_r & \\
t_r = t_x \Ra \bool & \\
t_x = \num  &  \\
t_c = \num & \\
t_f = \num \Ra t_c & \\
\hline
\end{array}$$

2. Next, process the first equation. It is $t_f = t_r$. Since at least one side involves a type variable, we turn it into a substitution: $t_f \mapsto t_r$. You can turn it into the subsitution $t_r \mapsto t_f$ as well. Both take you the same place. 


$$\begin{array}{rl}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
t_r = t_x \Ra \bool & t_r \mapsto t_f\\
t_x = \num  &  \\
t_c = \num & \\
t_f = \num \Ra t_c & \\
\hline
\end{array}$$

Substitutions are carried out lazily. I.e, we apply a substitution only when an equation is being processed.

3. Next, let us process the equation $t_r = t_x \Ra \bool$. Let us apply the substitutions to the LHS and RHS We get $t_f = t_x \Ra \bool$, by applying the substitution $t_r \mapsto t_f$ on the LHS. We can convert it to the rule $t_f \mapsto t_x \Ra \bool$.

$$\begin{array}{rl}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
\color{gray}{t_r = t_x \Ra \bool} & t_r \mapsto \color{red}{t_f}\\
t_x = \num  &  t_f  \mapsto t_x \Ra \bool \\
t_c = \num & \\
t_f = \num \Ra t_c & \\
\hline
\end{array}$$

__Note:__ $t_f$ appears on the RHS of a previous substitution. We will therefore substitute it out to maintain the following key property.


- **P1.** There can be at most one rule of the form $t \mapsto ...$ for each type variable $t$.
- **P2.** If $t$ appears in the LHS of some substitution rule, then it cannot appear in the RHS of any substitution rule.
- **P3.** Trivial substitutions such as $t \mapsto t$ should be discarded.
- **P4.** If a rule involves $t$ on both LHS and RHS, then we declare TYPE ERROR.

To make sure that **P2** is satisfied, we substitute for $t_f$.
$$\begin{array}{rl}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
t_x = \num  & t_r \mapsto (t_x \Ra \bool)\\
t_c = \num &  t_f  \mapsto t_x \Ra \bool \\
t_f = \num \Ra t_c & \\
\hline
\end{array}$$

3. Next let us process $t_x = \num$. It becomes a substitution rule $t_x \mapsto \num$.

$$\begin{array}{rl}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
\color{gray}{t_x = \num}  & t_r \mapsto (\color{red}{t_x} \Ra \bool)\\
t_c = \num &  t_f  \mapsto \color{red}{t_x} \Ra \bool \\
t_f = \num \Ra t_c & t_x \mapsto \num \\
\hline
\end{array}$$

Note that property **P2** is again violated. To fix it, we substitute again.

$$\begin{array}{rl}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
 t_c = \num  & t_r \mapsto \num \Ra \bool\\
t_f = \num \Ra t_c  &  t_f  \mapsto \num \Ra \bool \\
& t_x \mapsto \num \\
\hline
\end{array}$$

Next we process $t_c = \num$ as

$$\begin{array}{rl}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
  & t_r \mapsto \num \Ra \bool\\
t_f = \num \Ra t_c  &  t_f  \mapsto \num \Ra \bool \\
& t_x \mapsto \num \\
& t_c \mapsto \num \\
\hline
\end{array}$$

Last, we are left with the equation $t_f = \num \Ra t_c$.

We substitute, for $t_f$ on the LHS and $t_c$ on the RHS from ur substitution rules. The eq. is transformed into:
$$(\num \Ra \bool) = (\num \Ra \num) $$

This cannot be converted into a substitution since neither LHS or RHS is a type variable. Instead, we use
something called __unification__.
- **P5.** If one side of the Eq. is $\num, \bool$ and the other side is not the same, then we immediately conclude __type error__.
- **P6.** If both sides are funtion types $ (t_1 \Ra t_2) =  (t_3 \Ra t_4)$, we add the equations
$$ t_1 = t_3\ \text{and}\ t_2 = t_4 $$.

Returning to our example, since **P6** applies, we convert $(\num \Ra \bool) = (\num \Ra \num) $ into
$$ \begin{array}{c}
\num = \num\\
\bool = \num \\
\end{array}$$.

$$\begin{array}{rl}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
 \num = \num & t_r \mapsto \num \Ra \bool\\
\bool = \num &  t_f  \mapsto \num \Ra \bool \\
& t_x \mapsto \num \\
& t_c \mapsto \num \\
\hline
\end{array}$$

Let us process these equations. Following **P5** the eq. $\num = \num$ is thrown away. 

Next, we process $\num = \bool$. We conclude that these two cannot be equal, and therefore, we declare TYPE ERROR.

Let us try more examples.

### Example 2

Consider the following example:

~~~
let f = function (x) x + x in 
  f (f)
~~~


$$\begin{array}{rl}
\hline
\text{Expr./Ident} & \text{Type Variable} \\
\hline
\texttt{f} & t_f \\
\texttt{function (x) x + x} & t_r \\
\texttt{x} & t_x \\
\texttt{x+ x} & \num \\
\texttt{f(f)} & t_{ff}\\
\hline
\end{array}$$

We can now generate the equations.

$$\begin{array}{rl}
\hline
\text{Expr.} & \text{Eq.} \\
\hline
\texttt{let f = ... } & t_f = t_r \\
\texttt{function (x) x + x} & t_r = t_x \Ra \num \\
\texttt{x+ x} & t_x = \num \\
\texttt{f(f)} & t_f = t_f \Ra t_{ff}\\
\hline
\end{array}$$

Let us solve these equations.

$$\begin{array}{rl}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
\color{gray}{t_f = t_r} & t_f \mapsto t_r \\
 t_r = t_x \Ra \num & \\
 t_x = \num & \\
 t_f = t_f \Ra t_{ff}\ & \\
 \hline
 \end{array}$$
 
 $$\begin{array}{rll}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
\color{gray}{t_f = t_r} & t_f \mapsto \color{red}{t_r} \\
 \color{gray}{t_r = t_x \Ra \num} & t_r \mapsto t_x \Ra \num \\
 t_x = \num & \\
 t_f = t_f \Ra t_{ff}\ & \\
 \hline
 \end{array}$$
 
  $$\begin{array}{rl}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
\color{gray}{t_f = t_r} & t_f \mapsto \color{red}{t_r} \\
 \color{gray}{t_r = t_x} & t_x \mapsto \color{red}{t_r} \\
 \color{gray}{t_x = \num} & t_r \mapsto \num \\
 t_f = t_f \Ra t_{ff}\ & \\
 \hline
 \end{array}$$
 
  $$\begin{array}{rl}
\hline
\text{Equations} & \text{Substitutions} \\
\hline
\color{gray}{t_f = t_r} & t_f \mapsto \num \\
 \color{gray}{t_r = t_x} & t_x \mapsto \num \\
 \color{gray}{t_x = \num} & t_r \mapsto \num \\
 t_f = t_f \Ra t_{ff}\ & \\
 \hline
 \end{array}$$
 
 Let us now process the last equation. We obtain after substitutions:
 $$ \num = \num \Ra t_{ff}$$
 
 We note that the LHS is just $\num$ and RHS is a function type expression. We declare a TYPE ERROR.
 
 ## Example 3
 
 Let's look at a program that is actually well typed for a change.
 
 ~~~
 let f = function (g) 
             function (x) 
                 g( g( x ) )
         in 
      let d = function (y) y + y in 
          f (d)
 ~~~
 
 $$\begin{array}{rl}
 \hline
 \text{Expr/Ident} & \text{Type Variable}\\
 \hline
 f & t_f \\
 g & t_g  \\
 x & t_x \\
 \texttt{function (g)  function (x) g( g( x ) )} & t_{1} \\
 \texttt{function (x) g( g( x ) )} & t_2 \\
 \texttt{g(g(x))} & t_3 \\
 \texttt{g(x)} & t_4 \\
 d & t_d \\
 \texttt{function (y) y + y } & t_5 \\
 \texttt{y+y} & \num \\
 f(d) & t_6 \\
 \hline 
 \end{array}$$
 
 Let s write some equations:
 
 $$\begin{array}{rl}
 \hline
 \text{Expr/Ident} & \text{Type Eq}\\
 \hline
 \texttt{let f  = ... }& t_f = t_1\\
 \texttt{unction (g) ...} & t_1 = t_g \Ra t_2 \\
 \texttt{function (x) ...} & t_2 = t_x \Ra t_3 \\
 \texttt{g(g(x))} & t_g = t_4 \Ra t_3  \\
 \texttt{g(x)} & t_g = t_x \Ra t_4 \\
 \texttt{let d = function ..} & t_d = t_5\\
 \texttt{function (y) y + y } & t_5 = t_y \Ra \num \\
 \texttt{y+y} & t_y = \num \\
 f(d) & t_f = t_d \Ra t_6 \\
 \hline 
 \end{array}$$
 
 Let us solve the equations.
 
 $$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto t_1 \\
t_1 = t_g \Ra t_2 & \\
t_2 = t_x \Ra t_3 & \\
t_g = t_ 4 \Ra t_3 & \\
t_g = t_x \Ra t_4 & \\
t_d = t_5 & \\
t_5 = t_y \Ra \num & \\
t_y = \num  & \\
t_f = t_d \Ra t_6 & \\
\hline
\end{array}$$

$$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto t_g \Ra t_2 \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto t_g \Ra t_2\\
t_2 = t_x \Ra t_3 & \\
t_g = t_ 4 \Ra t_3 & \\
t_g = t_x \Ra t_4 & \\
t_d = t_5 & \\
t_5 = t_y \Ra \num & \\
t_y = \num  & \\
t_f = t_d \Ra t_6 & \\
\hline
\end{array}$$


$$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto t_g \Ra (t_x \Ra t_3) \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto t_g \Ra (t_x \Ra t_3)\\
\color{gray}{t_2 = t_x \Ra t_3} & t_2 \mapsto t_x \Ra t_3 \\
t_g = t_ 4 \Ra t_3 & \\
t_g = t_x \Ra t_4 & \\
t_d = t_5 & \\
t_5 = t_y \Ra \num & \\
t_y = \num  & \\
t_f = t_d \Ra t_6 & \\
\hline
\end{array}$$

$$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto (t_4 \Ra t_3) \Ra (t_x \Ra t_3) \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto (t_4 \Ra t_3) \Ra (t_x \Ra t_3)\\
\color{gray}{t_2 = t_x \Ra t_3} & t_2 \mapsto t_x \Ra t_3 \\
\color{gray}{t_g = t_4 \Ra t_3} & t_g \mapsto t_4 \Ra t_3\\
t_g = t_x \Ra t_4 & \\
t_d = t_5 & \\
t_5 = t_y \Ra \num & \\
t_y = \num  & \\
t_f = t_d \Ra t_6 & \\
\hline
\end{array}$$

Now consider the next eq. $ t_g = t_x \Ra t_4 $.
After substitutions to both sides, we convert it to $t_4 \Ra t_3 = t_x \Ra t_4 $.
In turn, this yields two equations
$$ t_4 = t_x,\ t_3 = t_4 $$.

$$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x) \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x)\\
\color{gray}{t_2 = t_x \Ra t_3} & t_2 \mapsto t_x \Ra t_x \\
\color{gray}{t_g = t_4 \Ra t_3} & t_g \mapsto t_x \Ra t_x\\
\color{gray}{t_g = t_x \Ra t_4} & \\
\color{gray}{t_4 = t_x} & t_4 \mapsto t_x\\
\color{gray}{t_3 = t_4} & t_3 \mapsto t_x \\
t_d = t_5 & \\
t_5 = t_y \Ra \num & \\
t_y = \num  & \\
t_f = t_d \Ra t_6 & \\
\hline
\end{array}$$

$$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x) \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x)\\
\color{gray}{t_2 = t_x \Ra t_3} & t_2 \mapsto t_x \Ra t_x \\
\color{gray}{t_g = t_4 \Ra t_3} & t_g \mapsto t_x \Ra t_x\\
\color{gray}{t_g = t_x \Ra t_4} & \\
\color{gray}{t_4 = t_x} & t_4 \mapsto t_x\\
\color{gray}{t_3 = t_4} & t_3 \mapsto t_x \\
\color{gray}{t_d = t_5 } & t_d\mapsto t_y \Ra \num \\
\color{gray}{t_5 = t_y \Ra \num} & t_5 \mapsto t_y \Ra \num\\
t_y = \num  & \\
t_f = t_d \Ra t_6 & \\
\hline
\end{array}$$

$$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x) \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x)\\
\color{gray}{t_2 = t_x \Ra t_3} & t_2 \mapsto t_x \Ra t_x \\
\color{gray}{t_g = t_4 \Ra t_3} & t_g \mapsto t_x \Ra t_x\\
\color{gray}{t_g = t_x \Ra t_4} & \\
\color{gray}{t_4 = t_x} & t_4 \mapsto t_x\\
\color{gray}{t_3 = t_4} & t_3 \mapsto t_x \\
\color{gray}{t_d = t_5 } & t_d\mapsto \num \Ra \num \\
\color{gray}{t_5 = t_y \Ra \num} & t_5 \mapsto \num\Ra \num\\
\color{gray}{t_y = \num}  & t_y \mapsto \num \\
t_f = t_d \Ra t_6 & \\
\hline
\end{array}$$

Now we are left with the last equation $t_f = t_d \Ra t_6$. Let us substitute the LHS and RHS.

$$(t_x \Ra t_x) \Ra (t_x \Ra t_x) = (\num \Ra \num) \Ra t_6 $$

Unifying, this yields,
$$ \begin{array}{c} t_x \Ra t_x = \num \Ra \num \\ t_x \Ra t_x = t_6 \end{array}$$

$$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x) \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x)\\
\color{gray}{t_2 = t_x \Ra t_3} & t_2 \mapsto t_x \Ra t_x \\
\color{gray}{t_g = t_4 \Ra t_3} & t_g \mapsto t_x \Ra t_x\\
\color{gray}{t_g = t_x \Ra t_4} & \\
\color{gray}{t_4 = t_x} & t_4 \mapsto t_x\\
\color{gray}{t_3 = t_4} & t_3 \mapsto t_x \\
\color{gray}{t_d = t_5 } & t_d\mapsto \num \Ra \num \\
\color{gray}{t_5 = t_y \Ra \num} & t_5 \mapsto \num\Ra \num\\
\color{gray}{t_y = \num}  & t_y \mapsto \num \\
\color{gray}{t_f = t_d \Ra t_6 }& \\
t_x \Ra t_x = \num \Ra \num & \\
t_x \Ra t_x = t_6 & \\
\hline
\end{array}$$

Now we process the next equation in our list: $t_x \Ra t_x = \num \Ra \num$, which requires unification and
yields the equations $t_x = \num$ and $t_x = \num$.

$$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x) \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x)\\
\color{gray}{t_2 = t_x \Ra t_3} & t_2 \mapsto t_x \Ra t_x \\
\color{gray}{t_g = t_4 \Ra t_3} & t_g \mapsto t_x \Ra t_x\\
\color{gray}{t_g = t_x \Ra t_4} & \\
\color{gray}{t_4 = t_x} & t_4 \mapsto t_x\\
\color{gray}{t_3 = t_4} & t_3 \mapsto t_x \\
\color{gray}{t_d = t_5 } & t_d\mapsto \num \Ra \num \\
\color{gray}{t_5 = t_y \Ra \num} & t_5 \mapsto \num\Ra \num\\
\color{gray}{t_y = \num}  & t_y \mapsto \num \\
\color{gray}{t_f = t_d \Ra t_6 }& \\
\color{gray}{t_x \Ra t_x = \num \Ra \num} & \\
t_x = \num & \\
t_x = \num & \\
t_x \Ra t_x = t_6 & \\
\hline
\end{array}$$

Processing $t_x = \num$, we obtain the substitution rule $t_x \mapsto \num$.


$$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto (\num \Ra \num) \Ra (\num \Ra \num) \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto (\num \Ra \num) \Ra (\num \Ra \num)\\
\color{gray}{t_2 = t_x \Ra t_3} & t_2 \mapsto \num \Ra \num \\
\color{gray}{t_g = t_4 \Ra t_3} & t_g \mapsto \num \Ra \num\\
\color{gray}{t_g = t_x \Ra t_4} & \\
\color{gray}{t_4 = t_x} & t_4 \mapsto \num\\
\color{gray}{t_3 = t_4} & t_3 \mapsto \num \\
\color{gray}{t_d = t_5 } & t_d\mapsto \num \Ra \num \\
\color{gray}{t_5 = t_y \Ra \num} & t_5 \mapsto \num\Ra \num\\
\color{gray}{t_y = \num}  & t_y \mapsto \num \\
\color{gray}{t_f = t_d \Ra t_6 }& \\
\color{gray}{t_x \Ra t_x = \num \Ra \num} & \\
\color{gray}{t_x = \num} & t_x \mapsto \num\\
t_x = \num & \\
t_x \Ra t_x = t_6 & \\
\hline
\end{array}$$

Processing $t_x = \num$, we substitute for both sides and get $\num = \num$. This is a trivial equation and is ignored.

Last, we process $t_x \Ra t_x = t_6$, which after subsitutions yields $\num \Ra \num = t_6$. The RHS is a type variable, and therefore, we turn it into a substitution:


$$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto (\num \Ra \num) \Ra (\num \Ra \num) \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto (\num \Ra \num) \Ra (\num \Ra \num)\\
\color{gray}{t_2 = t_x \Ra t_3} & t_2 \mapsto \num \Ra \num \\
\color{gray}{t_g = t_4 \Ra t_3} & t_g \mapsto \num \Ra \num\\
\color{gray}{t_g = t_x \Ra t_4} & \\
\color{gray}{t_4 = t_x} & t_4 \mapsto \num\\
\color{gray}{t_3 = t_4} & t_3 \mapsto \num \\
\color{gray}{t_d = t_5 } & t_d\mapsto \num \Ra \num \\
\color{gray}{t_5 = t_y \Ra \num} & t_5 \mapsto \num\Ra \num\\
\color{gray}{t_y = \num}  & t_y \mapsto \num \\
\color{gray}{t_f = t_d \Ra t_6 }& \\
\color{gray}{t_x \Ra t_x = \num \Ra \num} & \\
\color{gray}{t_x = \num} & t_x \mapsto \num\\
\color{gray}{t_x = \num} & \\
\color{gray}{t_x \Ra t_x = t_6} & t_6 \mapsto \num \Ra \num \\
\hline
\end{array}$$

As you can see, there are no more equations left and the types of the various unknown expressions have been computed.

### Example 3 (Polymorphism)



 ~~~
 let f = function (g) 
             function (x) 
                 g( g( x ) )
         in
          f 
 ~~~
 
 
 $$\begin{array}{rl}
 \hline
 \text{Expr/Ident} & \text{Type Variable}\\
 \hline
 f & t_f \\
 g & t_g  \\
 x & t_x \\
 \texttt{function (g)  function (x) g( g( x ) )} & t_{1} \\
 \texttt{function (x) g( g( x ) )} & t_2 \\
 \texttt{g(g(x))} & t_3 \\
 \texttt{g(x)} & t_4 \\
 \hline 
 \end{array}$$
 
 Let s write some equations:
 
 $$\begin{array}{rl}
 \hline
 \text{Expr/Ident} & \text{Type Eq}\\
 \hline
 \texttt{let f  = ... }& t_f = t_1\\
 \texttt{unction (g) ...} & t_1 = t_g \Ra t_2 \\
 \texttt{function (x) ...} & t_2 = t_x \Ra t_3 \\
 \texttt{g(g(x))} & t_g = t_4 \Ra t_3  \\
 \texttt{g(x)} & t_g = t_x \Ra t_4 \\
 \hline 
 \end{array}$$
 
 Let us solve the equations.
 
 $$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto (t_4 \Ra t_3) \Ra (t_x \Ra t_3) \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto (t_4 \Ra t_3) \Ra (t_x \Ra t_3)\\
\color{gray}{t_2 = t_x \Ra t_3} & t_2 \mapsto t_x \Ra t_3 \\
\color{gray}{t_g = t_4 \Ra t_3} & t_g \mapsto t_4 \Ra t_3\\
t_g = t_x \Ra t_4 & \\
\hline
\end{array}$$


Now consider the next eq. $ t_g = t_x \Ra t_4 $.
After substitutions to both sides, we convert it to $t_4 \Ra t_3 = t_x \Ra t_4 $.
In turn, this yields two equations
$$ t_4 = t_x,\ t_3 = t_4 $$.

When the dust settles: we have

 $$\begin{array}{rl}
 \hline
 \text{Eq.} & \text{Subst} \\
 \hline
\color{gray}{t_f = t_1} & t_f \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x) \\
\color{gray}{t_1 = t_g \Ra t_2} & t_1 \mapsto (t_x \Ra t_x) \Ra (t_x \Ra t_x)\\
\color{gray}{t_2 = t_x \Ra t_3} & t_2 \mapsto t_x \Ra t_x \\
\color{gray}{t_g = t_4 \Ra t_3} & t_g \mapsto t_x \Ra t_x\\
\color{gray}{t_g = t_x \Ra t_4} & \\
\color{gray}{t_4 = t_x} & t_4 \mapsto t_x \\
\color{gray}{t_3 = t_4 } & t_3 \mapsto t_x \\
\hline
\end{array}$$

We have no more type equations left to process and no type errors were discovered thus far. However,
notice how the types are not completely resolved yet. The program depends on the type of the formal
parameter $x$, which is not known. Whatever type we want for it works, but until we fix a type for
$x$, the program cannot be typed. 

Or to put a positive spin on it, the program could be executed for any type of $x$ and therefore
it is polymorphic.

Since our type checker did not allow polymorphic types, we simply revert to TYPE ERROR, which could
be resolved by the user adding an annotation only for the formal parameter $x$. Alternatively, 
a type annotation anywhere else for $f$ or formal parameter $g$ will also work.

## Conclusion (thus far)

We have shown how to generate equations for types and solve them systematically to compute the types or 
say that the program has a type error. However, we need to implement our algorithm in our interpreter.
Also, we need some means to report errors in a way that we can point the user out to where the error 
happened or in other words, the root cause. We will examine these topics in the subsequent lecture.